In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
#Carga desde un archivo .csv sin indice
df_filtered = pd.read_csv('airbnb_madrid_filtered.csv')
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26004 entries, 0 to 26003
Data columns (total 50 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              26004 non-null  int64  
 1   nombre                          26004 non-null  object 
 2   descripcion                     25104 non-null  object 
 3   descripcion_barrio              11158 non-null  object 
 4   id_anfitrion                    26004 non-null  int64  
 5   nombre_anfitrion                25989 non-null  object 
 6   anfitrion_desde                 25988 non-null  object 
 7   ubicacion_anfitrion             17935 non-null  object 
 8   tiempo_respuesta_anfitrion      21276 non-null  object 
 9   tasa_respuesta_anfitrion        21276 non-null  object 
 10  tasa_aceptacion_anfitrion       22019 non-null  object 
 11  es_superanfitrion               25049 non-null  object 
 12  anuncios_activos_anfitrion      

In [49]:
#Identificar valores nulos por columna
valores_nulos=df_filtered.isnull().sum()
valores_nulos

id                                    0
nombre                                0
descripcion                         900
descripcion_barrio                14846
id_anfitrion                          0
nombre_anfitrion                     15
anfitrion_desde                      16
ubicacion_anfitrion                8069
tiempo_respuesta_anfitrion         4728
tasa_respuesta_anfitrion           4728
tasa_aceptacion_anfitrion          3985
es_superanfitrion                   955
anuncios_activos_anfitrion           16
total_anuncios_anfitrion             16
verificaciones_anfitrion             16
anfitrion_tiene_foto                 16
identidad_anfitrion_verificada       16
barrio                                0
distrito                              0
latitud                               0
longitud                              0
tipo_propiedad                        0
tipo_habitacion                       0
capacidad                             0
banos                              5939


In [50]:
# 1. Eliminar columna con demasiados nulos
df_filtered = df_filtered.drop(columns=["descripcion_barrio"])

In [51]:
# Verificar duplicados en la columna 'id'
duplicados_id = df_filtered['id'].duplicated().sum()
print(f"Número de ids duplicados: {duplicados_id}")

Número de ids duplicados: 0


In [52]:

# Máscara para filas donde todas las columnas de "host_block" son nulas
host_block_cols = [
    "anuncios_activos_anfitrion",
    "total_anuncios_anfitrion",
    "verificaciones_anfitrion",
    "anfitrion_tiene_foto",
    "identidad_anfitrion_verificada",
]

mask_host_block = df_filtered[host_block_cols].isna().all(axis=1)



# Eliminar filas que cumplan cualquiera de las dos máscaras
df_filtered = df_filtered.loc[~(mask_host_block)].copy()
df_filtered.info()


<class 'pandas.core.frame.DataFrame'>
Index: 25988 entries, 0 to 26003
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              25988 non-null  int64  
 1   nombre                          25988 non-null  object 
 2   descripcion                     25088 non-null  object 
 3   id_anfitrion                    25988 non-null  int64  
 4   nombre_anfitrion                25988 non-null  object 
 5   anfitrion_desde                 25988 non-null  object 
 6   ubicacion_anfitrion             17935 non-null  object 
 7   tiempo_respuesta_anfitrion      21276 non-null  object 
 8   tasa_respuesta_anfitrion        21276 non-null  object 
 9   tasa_aceptacion_anfitrion       22019 non-null  object 
 10  es_superanfitrion               25034 non-null  object 
 11  anuncios_activos_anfitrion      25988 non-null  float64
 12  total_anuncios_anfitrion        25988

In [53]:
#Identificar valores nulos por columna
valores_nulos=df_filtered.isnull().sum()
valores_nulos

id                                   0
nombre                               0
descripcion                        900
id_anfitrion                         0
nombre_anfitrion                     0
anfitrion_desde                      0
ubicacion_anfitrion               8053
tiempo_respuesta_anfitrion        4712
tasa_respuesta_anfitrion          4712
tasa_aceptacion_anfitrion         3969
es_superanfitrion                  954
anuncios_activos_anfitrion           0
total_anuncios_anfitrion             0
verificaciones_anfitrion             0
anfitrion_tiene_foto                 0
identidad_anfitrion_verificada       0
barrio                               0
distrito                             0
latitud                              0
longitud                             0
tipo_propiedad                       0
tipo_habitacion                      0
capacidad                            0
banos                             5935
habitaciones                      2453
camas                    

In [54]:
# 1) Imputaciones simples (texto)
df_filtered["descripcion"] = df_filtered["descripcion"].fillna("Sin descripción")
df_filtered["ubicacion_anfitrion"] = df_filtered["ubicacion_anfitrion"].fillna("No reportada")
df_filtered["tiempo_respuesta_anfitrion"] = df_filtered["tiempo_respuesta_anfitrion"].fillna("Desconocida")


In [55]:
# 2) Imputaciones de disponibilidad y normalización a booleano en columna 'disponibilidad'

# True si es exactamente 't' (ignorando espacios y mayúsculas); en otro caso False
df_filtered["disponibilidad"] = (
    df_filtered["disponibilidad"]
      .fillna("")                # NaN -> ""
      .astype(str).str.strip().str.lower()
      .eq("t")                   # 't' -> True; cualquier otro -> False
)

#  verificamos
print(df_filtered["disponibilidad"].dtype)   # bool
print(df_filtered["disponibilidad"].value_counts(dropna=False))



bool
disponibilidad
True     24471
False     1517
Name: count, dtype: int64


In [56]:
# Elimina filas donde:
# - tasa_aceptacion_anfitrion está vacía
# - tasa_respuesta_anfitrion está vacía
# - primera_resena está vacía
# (vacía = NaN o cadena "")

mask_drop = (
    (df_filtered["tasa_aceptacion_anfitrion"].isna() | df_filtered["tasa_aceptacion_anfitrion"].astype(str).str.strip().eq(""))
    &
    (df_filtered["tasa_respuesta_anfitrion"].isna()  | df_filtered["tasa_respuesta_anfitrion"].astype(str).str.strip().eq(""))
    &
    (df_filtered["primera_resena"].isna()            | df_filtered["primera_resena"].astype(str).str.strip().eq(""))
)

# Aplicar eliminación
rows_before = len(df_filtered)
df_filtered = df_filtered.loc[~mask_drop].copy()
print(f"Filas eliminadas: {mask_drop.sum()} / {rows_before}")


Filas eliminadas: 1657 / 25988


In [57]:
#Identificar valores nulos por columna
valores_nulos=df_filtered.isnull().sum()
valores_nulos

id                                   0
nombre                               0
descripcion                          0
id_anfitrion                         0
nombre_anfitrion                     0
anfitrion_desde                      0
ubicacion_anfitrion                  0
tiempo_respuesta_anfitrion           0
tasa_respuesta_anfitrion          3055
tasa_aceptacion_anfitrion         2312
es_superanfitrion                  949
anuncios_activos_anfitrion           0
total_anuncios_anfitrion             0
verificaciones_anfitrion             0
anfitrion_tiene_foto                 0
identidad_anfitrion_verificada       0
barrio                               0
distrito                             0
latitud                              0
longitud                             0
tipo_propiedad                       0
tipo_habitacion                      0
capacidad                            0
banos                             4527
habitaciones                      1732
camas                    

In [58]:
# 3) Eliminar filas sin info de reseñas (todas las columnas de reseñas vacías/NaN)
# Definir columnas de reseñas
cols_reviews = [
    "primera_resena",
    "ultima_resena",
    "puntaje_total",
    "puntaje_limpieza",
    "puntaje_comunicacion",
    "puntaje_ubicacion",
]


df_filtered[cols_reviews] = df_filtered[cols_reviews].replace(r"^\s*$", np.nan, regex=True)

# Mask para filas donde todas las columnas de reseñas son nulas
mask_drop_reviews = df_filtered[cols_reviews].isna().all(axis=1)

# Aplicar eliminación
rows_before = len(df_filtered)
df_filtered = df_filtered.loc[~mask_drop_reviews].copy()
print(f"Filas eliminadas (sin info de reseñas): {mask_drop_reviews.sum()} / {rows_before}")



Filas eliminadas (sin info de reseñas): 3580 / 24331


In [59]:
#Identificar valores nulos por columna
valores_nulos=df_filtered.isnull().sum()
valores_nulos

id                                   0
nombre                               0
descripcion                          0
id_anfitrion                         0
nombre_anfitrion                     0
anfitrion_desde                      0
ubicacion_anfitrion                  0
tiempo_respuesta_anfitrion           0
tasa_respuesta_anfitrion          2882
tasa_aceptacion_anfitrion         2247
es_superanfitrion                  901
anuncios_activos_anfitrion           0
total_anuncios_anfitrion             0
verificaciones_anfitrion             0
anfitrion_tiene_foto                 0
identidad_anfitrion_verificada       0
barrio                               0
distrito                             0
latitud                              0
longitud                             0
tipo_propiedad                       0
tipo_habitacion                      0
capacidad                            0
banos                             3945
habitaciones                      1582
camas                    

In [60]:
df_filtered["precio"] = df_filtered["precio"].replace('[\$,]', '', regex=True).astype(float)
df_filtered["ingresos_estimados_365d"] = df_filtered["ingresos_estimados_365d"].astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\ISAAC\AppData\Local\Temp\ipykernel_27156\4177539821.py:1: SyntaxWarning: invalid escape sequence '\$'
  df_filtered["precio"] = df_filtered["precio"].replace('[\$,]', '', regex=True).astype(float)


In [61]:
# verificar skew para decidir imputación por media o mediana para 'precio' e 'ingresos_estimados_365d'
skew_precio = df_filtered["precio"].dropna().skew()
skew_rev   = df_filtered["ingresos_estimados_365d"].dropna().skew()
print(f"Skew precio: {skew_precio:.2f} | Skew ingresos: {skew_rev:.2f}")
# Regla práctica: si |skew| > 1, evita la media y usa mediana/derivado.

Skew precio: 43.34 | Skew ingresos: 19.68


In [65]:
# Imputación IN-PLACE con mediana por cascada de segmentos
# Columnas float64: 'precio' y 'ingresos_estimados_365d'

cols = ["precio", "ingresos_estimados_365d"]

for c in cols:
    # 1) distrito + tipo_habitacion
    med1 = df_filtered.groupby(["distrito", "tipo_habitacion"], dropna=False)[c].transform("median")
    df_filtered[c] = df_filtered[c].fillna(med1)

    # 2) solo distrito
    med2 = df_filtered.groupby("distrito", dropna=False)[c].transform("median")
    df_filtered[c] = df_filtered[c].fillna(med2)

    # 3) solo tipo_habitacion
    med3 = df_filtered.groupby("tipo_habitacion", dropna=False)[c].transform("median")
    df_filtered[c] = df_filtered[c].fillna(med3)

    # 4) fallback global
    df_filtered[c] = df_filtered[c].fillna(df_filtered[c].median())
df_filtered[["precio", "ingresos_estimados_365d"]].head(15)

,precio,ingresos_estimados_365d
0,29.0,0.0
1,135.0,17595.0
2,135.0,17595.0
3,135.0,17595.0
4,64.0,13056.0
5,86.0,2408.0
6,196.0,21168.0
7,143.5,14631.0
8,72.0,0.0
9,209.0,33858.0


In [67]:
# Imputar con la media los puntajes de limpieza, comunicación y ubicación
cols_scores = ["puntaje_limpieza", "puntaje_comunicacion", "puntaje_ubicacion"]

for c in cols_scores:
    df_filtered[c] = df_filtered[c].fillna(df_filtered[c].mean())


In [68]:
#Identificar valores nulos por columna
valores_nulos=df_filtered.isnull().sum()
valores_nulos

id                                   0
nombre                               0
descripcion                          0
id_anfitrion                         0
nombre_anfitrion                     0
anfitrion_desde                      0
ubicacion_anfitrion                  0
tiempo_respuesta_anfitrion           0
tasa_respuesta_anfitrion          2882
tasa_aceptacion_anfitrion         2247
es_superanfitrion                  901
anuncios_activos_anfitrion           0
total_anuncios_anfitrion             0
verificaciones_anfitrion             0
anfitrion_tiene_foto                 0
identidad_anfitrion_verificada       0
barrio                               0
distrito                             0
latitud                              0
longitud                             0
tipo_propiedad                       0
tipo_habitacion                      0
capacidad                            0
banos                             3945
habitaciones                      1582
camas                    

In [71]:

def impmedia(df, col, levels, min_count=5):
    """
    Imputa EN LA MISMA COLUMNA usando la MEDIA por niveles de segmentación.
    levels: lista de listas con claves de agrupación (de más específico a más general).
    min_count: tamaño mínimo del grupo para usar su media; si no llega, pasa al siguiente nivel.
    """
    x = df[col].copy()
    for keys in levels:
        grp = df.groupby(keys, dropna=False)[col]
        mean = grp.transform("mean")
        cnt  = grp.transform("count")
        mask = x.isna() & (cnt >= min_count)
        x.loc[mask] = mean.loc[mask]
    # Fallback global
    x = x.fillna(df[col].mean())
    df[col] = x  # in-place

# Jerarquía de segmentación (de más específico a más general)
levels = [
    ["barrio",   "tipo_propiedad", "tipo_habitacion", "capacidad"],
    ["distrito", "tipo_propiedad", "tipo_habitacion", "capacidad"],
    ["distrito", "tipo_propiedad", "tipo_habitacion"],
    ["distrito", "tipo_habitacion"],
    ["tipo_propiedad", "tipo_habitacion"],
    ["tipo_habitacion"],
]

# --- 1) Imputar por MEDIA segmentada ---
for c in ["banos", "habitaciones", "camas"]:
    impmedia(df_filtered, c, levels, min_count=5)

# --- 2) Enforzar valores válidos por variable ---

# a) baños: solo múltiplos de 0.5
df_filtered["banos"] = (df_filtered["banos"] * 2).round() / 2.0
df_filtered["banos"] = df_filtered["banos"].clip(lower=0)  # por seguridad

# b) habitaciones y camas: solo enteros
for c in ["habitaciones", "camas"]:
    df_filtered[c] = df_filtered[c].round().astype(int)
    df_filtered[c] = df_filtered[c].clip(lower=0)  # por seguridad

df_filtered[["banos", "habitaciones", "camas"]].head(15)

,banos,habitaciones,camas
0,1.0,1,1
1,1.0,1,1
2,1.0,1,1
3,1.0,1,1
4,1.5,1,2
5,1.0,1,1
6,1.0,3,5
7,1.0,1,1
8,1.0,1,1
9,1.0,3,3


In [72]:
#Identificar valores nulos por columna
valores_nulos=df_filtered.isnull().sum()
valores_nulos

id                                   0
nombre                               0
descripcion                          0
id_anfitrion                         0
nombre_anfitrion                     0
anfitrion_desde                      0
ubicacion_anfitrion                  0
tiempo_respuesta_anfitrion           0
tasa_respuesta_anfitrion          2882
tasa_aceptacion_anfitrion         2247
es_superanfitrion                  901
anuncios_activos_anfitrion           0
total_anuncios_anfitrion             0
verificaciones_anfitrion             0
anfitrion_tiene_foto                 0
identidad_anfitrion_verificada       0
barrio                               0
distrito                             0
latitud                              0
longitud                             0
tipo_propiedad                       0
tipo_habitacion                      0
capacidad                            0
banos                                0
habitaciones                         0
camas                    

In [73]:
# es_superanfitrion: NaN -> "No especificado"; "t" -> "Si"; "f" -> "No"
s = df_filtered["es_superanfitrion"]

# Normalizamos para comparar contra 't'/'f' sin importar espacios o mayúsculas
snorm = s.astype(str).str.strip().str.lower()

# Sustituciones solicitadas
sust = s.copy()
sust.loc[snorm == "t"] = "Si"
sust.loc[snorm == "f"] = "No"

# Imputación de NAs
sust = sust.fillna("No especificado")

df_filtered["es_superanfitrion"] = sust


In [80]:
import re

def parse_pct_inplace(series: pd.Series) -> pd.Series:
    """Convierte '98%' -> 98.0; maneja 'No reportada', vacíos, 0–1 como proporción."""
    def _p(s):
        if pd.isna(s): 
            return np.nan
        s = str(s).strip().replace("%", "").replace(",", ".")
        s = re.sub(r"[^0-9.\-]", "", s)  # deja solo dígitos y punto
        if s == "" or s == ".": 
            return np.nan
        try:
            v = float(s)
        except:
            return np.nan
        # si vino como proporción 0–1, pásalo a 0–100
        if 0 <= v <= 1:
            v *= 100
        return float(np.clip(v, 0, 100))
    return series.apply(_p).astype("float64")

# 1) Parsear in-place a float (0–100)
df_filtered["tasa_respuesta_anfitrion"]  = parse_pct_inplace(df_filtered["tasa_respuesta_anfitrion"])
df_filtered["tasa_aceptacion_anfitrion"] = parse_pct_inplace(df_filtered["tasa_aceptacion_anfitrion"])

# 2) Imputar por la MEDIA (global) in-place
for c in ["tasa_respuesta_anfitrion", "tasa_aceptacion_anfitrion"]:
    df_filtered[c] = df_filtered[c].fillna(df_filtered[c].mean())

In [81]:
#Identificar valores nulos por columna
valores_nulos=df_filtered.isnull().sum()
valores_nulos

id                                0
nombre                            0
descripcion                       0
id_anfitrion                      0
nombre_anfitrion                  0
anfitrion_desde                   0
ubicacion_anfitrion               0
tiempo_respuesta_anfitrion        0
tasa_respuesta_anfitrion          0
tasa_aceptacion_anfitrion         0
es_superanfitrion                 0
anuncios_activos_anfitrion        0
total_anuncios_anfitrion          0
verificaciones_anfitrion          0
anfitrion_tiene_foto              0
identidad_anfitrion_verificada    0
barrio                            0
distrito                          0
latitud                           0
longitud                          0
tipo_propiedad                    0
tipo_habitacion                   0
capacidad                         0
banos                             0
habitaciones                      0
camas                             0
amenidades                        0
precio                      

In [82]:
#Convertir DataFrame a CSV
df_filtered.to_csv("Airbnb_Madrid_SinNulos.csv")

In [4]:
# ---- Reporte: Manejo de Valores Nulos (Airbnb Madrid) ----
reporte_nulos_md = """
# Manejo de Valores Nulos – Resumen Metodológico

 Objetivo
Estandarizar y completar la información faltante del dataset de Airbnb Madrid para habilitar análisis confiables.

 1) Eliminaciones de filas (criterios estrictos)
- Bloque inactivo del host: se eliminaron filas donde todas las columnas del perfil del anfitrión estaban vacías (anuncios_activos_anfitrion, total_anuncios_anfitrion, verificaciones_anfitrion, anfitrion_tiene_foto, identidad_anfitrion_verificada).  
  Justificación: alta probabilidad de inactividad/no operatividad.
- Inactividad de mercado: se eliminaron filas con num_resenas == 0 y (disponibilidad ∈ {“no”, “false”, “0”} o availability_365d == 0).  
  Justificación: listados sin tracción ni disponibilidad informativa.

(Adicionalmente, se eliminaron filas sin cualquier traza de reseñas: primera_resena, ultima_resena, puntaje_total, puntaje_limpieza, puntaje_comunicacion, puntaje_ubicacion todos vacíos.)

 2) Normalización y tipificación
- Disponibilidad: columna booleana; solo "t" se interpreta como True; vacíos u otros valores ⇒ False.  
- Superhost: imputación de vacíos como "No especificado" y mapeo "t"→"Si", "f"→"No".  
- Tasas del host: tasa_respuesta_anfitrion y tasa_aceptacion_anfitrion se parsearon a float (0–100) desde formatos tipo "98%".

 3) Imputación (reglas por tipo de variable)
- Texto  
  - descripcion → “Sin descripción”.  
  - ubicacion_anfitrion → “No reportada”.  
  - tiempo_respuesta_anfitrion → “Desconocida”.  
  Justificación: mantiene consistencia semántica sin inventar contenido.

- Tasas del host (numéricas 0–100)  
  - Imputación por media (global) en la misma columna tras parseo.  
  Justificación: variables acotadas; la media es estable y facilita comparabilidad. 

- Estructura del inmueble: banos, habitaciones, camas  
  - Imputación por media segmentada en cascada con la jerarquía:  
    barrio → distrito → (tipo_propiedad, tipo_habitacion) → tipo_habitacion, considerando también capacidad.  
  - Reglas de dominio post-imputación:  
    - banos solo múltiplos de 0.5 (p. ej., 1.0, 1.5, 2.0).  
    - habitaciones y camas solo enteros.  
  Justificación: respetar la realidad operativa del inventario; la segmentación reduce sesgo entre zonas/tipos.

- Precio e Ingresos  
  - Precio (precio): imputación por mediana/segmento en cascada (dada su fuerte asimetría).  
  - Ingresos estimados (ingresos_estimados_365d): imputación por mediana/segmento (o derivación con precio × ocupación cuando aplica).  
  Justificación: distribuciones muy sesgadas; la mediana es más robusta que la media.

- Puntajes (0–5): puntaje_limpieza, puntaje_comunicacion, puntaje_ubicacion  
  - Imputación por media (global) en la misma columna.  
  Justificación: la media mantiene coherencia de escala.

 4) Buenas prácticas aplicadas
- Respeto del dominio: baños en incrementos de 0.5; habitaciones/camas enteras; porcentajes 0–100; booleanos normalizados.  
- Imputación in-place y segmentada: se imputó directamente en las columnas, usando contexto de mercado cuando corresponde (barrio/distrito/tipo/capacidad).   

 5) Impacto esperado
- Mayor completitud sin comprometer la validez estadística.  
- Comparabilidad entre segmentos (por tipo y zona) manteniendo la estructura real del mercado.  
- Mejor base para análisis descriptivos, visualizaciones y modelos (con bajo sesgo por outliers y reglas de dominio respetadas).

"""

print(reporte_nulos_md)




# Manejo de Valores Nulos – Resumen Metodológico

 Objetivo
Estandarizar y completar la información faltante del dataset de Airbnb Madrid para habilitar análisis confiables.

 1) Eliminaciones de filas (criterios estrictos)
- Bloque inactivo del host: se eliminaron filas donde todas las columnas del perfil del anfitrión estaban vacías (anuncios_activos_anfitrion, total_anuncios_anfitrion, verificaciones_anfitrion, anfitrion_tiene_foto, identidad_anfitrion_verificada).  
  Justificación: alta probabilidad de inactividad/no operatividad.
- Inactividad de mercado: se eliminaron filas con num_resenas == 0 y (disponibilidad ∈ {“no”, “false”, “0”} o availability_365d == 0).  
  Justificación: listados sin tracción ni disponibilidad informativa.

(Adicionalmente, se eliminaron filas sin cualquier traza de reseñas: primera_resena, ultima_resena, puntaje_total, puntaje_limpieza, puntaje_comunicacion, puntaje_ubicacion todos vacíos.)

 2) Normalización y tipificación
- Disponibilidad: columna b